# Графовая база данных Neo4J

 ### Авиарасписания

Рубанова

Суворова


In [1]:
from neo4j import GraphDatabase, basic_auth
import ipywidgets as widgets
import re

In [2]:
driver = GraphDatabase.driver(uri = "bolt://localhost:7687", basic_auth=("neo4j", "neo4j"), encrypted=False)

In [3]:
aircraft = driver.session().run('LOAD CSV WITH HEADERS FROM "file:///aircraft_types.csv" AS row CREATE (n:Aircraft) SET n = row, n.aircraft_type = toString(row.aircraft_type), n.manufacturer = toString(row.manufacturer), n.capacity = toInteger(row.capacity), n.body_type = toString(row.body_type), n.distance = toString(row.distance)')

In [4]:
airline = driver.session().run('LOAD CSV WITH HEADERS FROM "file:///airlines.csv" AS row CREATE (n:Airline) SET n = row, n.IATA = toString(row.IATA), n.name = toString(row.name), n.country = toString(row.country), n.fleet_size = toInteger(row.fleet_size), n.destinations = toInteger(row.destinations)')

In [5]:
airport = driver.session().run('LOAD CSV WITH HEADERS FROM "file:///airports.csv" AS row CREATE (n:Airport) SET n = row, n.iata_code = toString(row.iata_code), n.city = toString(row.city), n.country = toString(row.country)')

In [6]:
city = driver.session().run('LOAD CSV WITH HEADERS FROM "file:///cities.csv" AS row CREATE (n:City) SET n = row, n.city_name = toString(row.city_name), n.country = toString(row.country), n.capital = toString(row.capital), n.nature = toString(row.nature), n.city_interest = toString(row.city_interest)')

In [7]:
timetable = driver.session().run('LOAD CSV WITH HEADERS FROM "file:///timetable.csv" AS row CREATE (n:Timetable) SET n = row, n.airline = toString(row.airline), n.flight_number = toString(row.flight_number), n.departure_time = toString(row.departure_time), n.arrival_time = toString(row.arrival_time), n.departure_airport = toString(row.departure_airport), n.destination_airport = toString(row.destination_airport), n.airplane_model = toString(row.airplane_model), n.flight_duration = toString(row.flight_duration)')

In [8]:
aircraft_type = driver.session().run('MATCH (t:Timetable),(a:Aircraft) WHERE t.airplane_model = a.aircraft_type CREATE (t)-[:aircraft_type]->(a)')

In [9]:
driver.session().run('MATCH (a:Airport),(c:City) WHERE a.city = c.city_name CREATE (a)-[:located_in]->(c)')

In [10]:
driver.session().run('MATCH (t:Timetable),(a:Airport) WHERE t.departure_airport = a.iata_code CREATE (t)-[:flies_from]->(a)')

In [11]:
driver.session().run('MATCH (t:Timetable),(a:Airport) WHERE t.destination_airport = a.iata_code CREATE (t)-[:flies_to]->(a)')

In [12]:
driver.session().run('MATCH (t:Timetable),(a:Airline) WHERE t.airline = a.IATA CREATE (t)-[:operated_by]->(a)')

# UPDATE

In [16]:
table = widgets.Dropdown(
    options=['City', 'Airport', 'Aircraft', 'Flight', 'Airline'])

print('Выберите тип объекта')
table

Выберите тип объекта


Dropdown(options=('City', 'Airport', 'Aircraft', 'Flight', 'Airline'), value='City')

In [86]:
if table.value == 'Flight':
    category = 'Timetable'
else:
    category = table.value

In [87]:
get_props = 'MATCH (n:'+category+') RETURN DISTINCT keys(n), size(keys(n)) ORDER BY size(keys(n)) DESC'

In [88]:
props = driver.session().run(get_props)
properties = props.value()[0]

In [15]:
class date_input():
    def __init__(self):
        
        self.input_text_column = widgets.Dropdown(
    options=properties,
    description='признак')
        self.input_text_set_value = widgets.Text(description = 'задать как')  
        self.input_text_condition_column = widgets.Dropdown(
    options=properties,
    description='если признак')
        self.input_text_condition_value = widgets.Text(description = 'содержит значение:')  

        self.input_text_set_value.on_submit(self.handle_submit)
        self.input_text_condition_value.on_submit(self.handle_submit)
        
        display(self.input_text_column, self.input_text_set_value, self.input_text_condition_column,
                self.input_text_condition_value)

    def handle_submit(self, text):
        self.v = text.value
        return self.v

f = date_input()

NameError: name 'properties' is not defined

In [90]:
query = 'MATCH (n {'+f.input_text_condition_column.value+ ':"'+f.input_text_condition_value.value+'" }) SET n.'+f.input_text_column.value+' ="'+f.input_text_set_value.value+'"'

In [91]:
query

'MATCH (n {city_name:"Moscow" }) SET n.nature ="mountain"'

In [92]:
driver.session().run(query)

# DELETE

In [78]:
options = ['Aircraft', 'Airport', 'Airline', 'City', 'Timetable']

table = widgets.Dropdown(
    options=options)

print('Выберите тип объекта, который вы хотите удалить: ')
table

Выберите тип объекта, который вы хотите удалить: 


Dropdown(options=('Aircraft', 'Airport', 'Airline', 'City', 'Timetable'), value='Aircraft')

In [141]:
def get_columns(t):
    res = driver.session().run('MATCH (n:{}) RETURN DISTINCT keys(n), size(keys(n)) ORDER BY size(keys(n)) DESC'.format(t))
    res = res.value()[0]
    return res

In [142]:
type_to_delete = table.value
columns = get_columns(type_to_delete)
columns

['flight_number',
 'flight_duration',
 'destination_airport',
 'departure_time',
 'arrival_time',
 'departure_airport',
 'airline',
 'airplane_model']

In [143]:
def get_options(i):
    res = driver.session().run('MATCH (c:{}) RETURN c.{}'.format(type_to_delete, i))
    res = set(res.value())
    return res

In [144]:
d = {}
for i in columns:
    values = get_options(i)
    d[i] = values

In [145]:
d

{'flight_number': {'1317',
  '2263',
  '278',
  '3661',
  '509',
  '574',
  '610',
  '7033',
  '845',
  '944'},
 'flight_duration': {'00:46:00',
  '01:25:00',
  '01:45:00',
  '02:20:00',
  '02:30:00',
  '02:45:00',
  '03:45:00',
  '04:25:00',
  '04:45:00',
  '04:55:00'},
 'destination_airport': {'AYT',
  'BCN',
  'DME',
  'GOJ',
  'KZN',
  'MSQ',
  'SVO',
  'TXL'},
 'departure_time': {'01:30:00',
  '04:20:00',
  '05:35:00',
  '09:40:00',
  '10:05:00',
  '12:55:00',
  '13:20:00',
  '18:50:00',
  '21:20:00',
  '21:40:00'},
 'arrival_time': {'00:05:00',
  '00:10:00',
  '00:15:00',
  '05:06:00',
  '05:15:00',
  '07:00:00',
  '10:35:00',
  '12:50:00',
  '14:05:00',
  '15:15:00'},
 'departure_airport': {'AER',
  'BAX',
  'DME',
  'GOJ',
  'LED',
  'MMK',
  'TLL',
  'TLV',
  'VKO'},
 'airline': {'B2', 'DP', 'RT', 'S7', 'SU', 'U6', 'WZ'},
 'airplane_model': {'A 320',
  'A 320-200',
  'A 321',
  'B 735',
  'B 737-800',
  'CRJ 100-200',
  'CRJ 200',
  'SSJ 100-95'}}

In [146]:
arr = []
for k, v in d.items():
    w = widgets.Dropdown(options=v, description=k)
    display(w)
    arr.append(w)

Dropdown(description='flight_number', options=('574', '2263', '1317', '610', '278', '509', '944', '845', '7033…

Dropdown(description='flight_duration', options=('02:45:00', '01:45:00', '04:25:00', '03:45:00', '04:55:00', '…

Dropdown(description='destination_airport', options=('KZN', 'SVO', 'DME', 'MSQ', 'AYT', 'TXL', 'GOJ', 'BCN'), …

Dropdown(description='departure_time', options=('01:30:00', '05:35:00', '04:20:00', '10:05:00', '09:40:00', '1…

Dropdown(description='arrival_time', options=('05:15:00', '12:50:00', '15:15:00', '07:00:00', '00:05:00', '10:…

Dropdown(description='departure_airport', options=('BAX', 'VKO', 'LED', 'DME', 'AER', 'TLV', 'TLL', 'GOJ', 'MM…

Dropdown(description='airline', options=('U6', 'B2', 'S7', 'DP', 'RT', 'WZ', 'SU'), value='U6')

Dropdown(description='airplane_model', options=('A 320', 'CRJ 200', 'A 321', 'CRJ 100-200', 'B 737-800', 'SSJ …

In [147]:
values = ["'"+str(item.value)+"'" for item in arr]
prodelete = dict(zip(columns, values))

In [148]:
prodelete

{'flight_number': "'2263'",
 'flight_duration': "'01:45:00'",
 'destination_airport': "'SVO'",
 'departure_time': "'21:20:00'",
 'arrival_time': "'00:05:00'",
 'departure_airport': "'TLL'",
 'airline': "'SU'",
 'airplane_model': "'SSJ 100-95'"}

In [149]:
pr = str(prodelete)
prod = re.sub("'", '', pr)
del_query = 'MATCH (a:'+table.value+' '+prod+') DETACH DELETE a'

In [150]:
del_query

'MATCH (a:Timetable {flight_number: "2263", flight_duration: "01:45:00", destination_airport: "SVO", departure_time: "21:20:00", arrival_time: "00:05:00", departure_airport: "TLL", airline: "SU", airplane_model: "SSJ 100-95"}) DETACH DELETE a'

In [151]:
driver.session().run(del_query)

# CREATE

In [93]:
table = widgets.Dropdown(
    options=['City', 'Airport', 'Aircraft', 'Flight', 'Airline'])

print('Выберите тип объекта')
table

Выберите тип объекта


Dropdown(options=('City', 'Airport', 'Aircraft', 'Flight', 'Airline'), value='City')

In [95]:
if table.value == 'Flight':
    category = 'Timetable'
else:
    category = table.value
    
get_props = 'MATCH (n:'+category+') RETURN DISTINCT keys(n), size(keys(n)) ORDER BY size(keys(n)) DESC'

props = driver.session().run(get_props)
properties = props.value()[0]

In [99]:
items = [widgets.Text(description = pro) for pro in properties]
for i in items:
    display(i)

Text(value='', description='aircraft_type')

Text(value='', description='manufacturer')

Text(value='', description='distance')

Text(value='', description='body_type')

Text(value='', description='capacity')

In [105]:
values = ["'"+item.value+"'" for item in items]
procreate = dict(zip(properties, values))

In [107]:
pr = str(procreate)

In [110]:
procr = re.sub("'", '', pr)

In [111]:
procr

'{aircraft_type: "a", manufacturer: "s", distance: "d", body_type: "f", capacity: "g"}'

In [112]:
creation_query  = 'CREATE (a:'+table.value+' '+procr+')'

In [113]:
creation_query

'CREATE (a:Aircraft {aircraft_type: "a", manufacturer: "s", distance: "d", body_type: "f", capacity: "g"})'

In [35]:
driver.session().run(creation_query)

NameError: name 'creation_query' is not defined

# **READ**

Задача: посмотреть какое-то свойство  по всем объектам


Например, тип - аэропорт, свойство - город

Видим все города, где есть аэропорты (по нашей базе)


Например, тип - самолет, свойство - тип

Все модели самолетов


Например, тип - рейс, свойство - номер рейса

все номера рейсов :)

In [91]:
table = widgets.Dropdown(
    options=['City', 'Airport', 'Aircraft', 'Flight', 'Airline'])

print('Выберите тип объекта')
table

Выберите тип объекта


Dropdown(options=('City', 'Airport', 'Aircraft', 'Flight', 'Airline'), value='City')

In [92]:
if table.value == 'Flight':
    category = 'Timetable'
else:
    category = table.value

In [93]:
get_props = 'MATCH (n:'+category+') RETURN DISTINCT keys(n), size(keys(n)) ORDER BY size(keys(n)) DESC'

props = driver.session().run(get_props)
properties = props.value()[0]

In [94]:
what_to_show = widgets.Dropdown(
    options= properties)

print('какую информацию об объекте вы хотите узнать')
what_to_show

какую информацию об объекте вы хотите узнать


Dropdown(options=('nature', 'city_interest', 'capital', 'city_name', 'country'), value='nature')

In [95]:
query_info = 'MATCH (n:'+category+') RETURN n.'+what_to_show.value
run = driver.session().run(query_info)
result = run.value()

In [96]:
result

['vacation',
 'vacation',
 'vacation',
 'culture',
 'cuture',
 'business',
 'culture',
 'culture',
 'culture',
 'culture',
 'vacation',
 'culture',
 'vacation']

# SORT

показать рейсы, отсортированные по времени (или чему-то другому)

In [26]:
table = widgets.Dropdown(
    options=['City', 'Airport', 'Aircraft', 'Flight', 'Airline'])

print('Выберите тип объекта')
table

Выберите тип объекта


Dropdown(options=('City', 'Airport', 'Aircraft', 'Flight', 'Airline'), value='City')

In [153]:
if table.value == 'Flight':
    category = 'Timetable'
else:
    category = table.value

In [154]:
get_props = 'MATCH (n:'+category+') RETURN DISTINCT keys(n), size(keys(n)) ORDER BY size(keys(n)) DESC'

props = driver.session().run(get_props)
properties = props.value()[0]

In [162]:
what_to_show = widgets.SelectMultiple(                ###чтобы выбрать несколько признаков, зажми CTRL и выбирай
    options = properties)                             ### UPD: несколько признаков не отражаются в результате запроса, почему-то
                                                    ### а на сервере с таким же запросом - отражаются
print('какую информацию об объекте вы хотите узнать')
what_to_show

какую информацию об объекте вы хотите узнать


SelectMultiple(options=('iata_code', 'city'), value=())

In [163]:
to_join = []

for k in what_to_show.value:
    to_join.append('n.'+k)

to_query = ', '.join(to_join)

In [164]:
sort_char = widgets.RadioButtons(                        ###чтобы выбрать несколько признаков, зажми CTRL и выбирай
    options = properties)

print('сортировать по')
sort_char

сортировать по


RadioButtons(options=('iata_code', 'city'), value='iata_code')

In [165]:
sort_query ='MATCH (n:'+category+') RETURN '+to_query+' ORDER BY n.'+sort_char.value+' ASC'

In [166]:
sort_query

'MATCH (n:Airport) RETURN n.city ORDER BY n.iata_code ASC'

In [167]:
run = driver.session().run(sort_query)
sorted_result = run.value()

In [168]:
sorted_result

['Sochi',
 'Antalya',
 'Barnaul',
 'Barcelona',
 'Moscow',
 'Nizhny_Novgorod',
 'Saint_Petersburg',
 'Murmansk',
 'Minsk',
 'Moscow',
 'Tallinn',
 'Tel_Aviv',
 'Berlin']

# AGGREGATING FUNCTIONS

# 1. Collect

Узнать все "входящие" рейсы в какой-то аэропорт.

In [85]:
def get_options(i):
    res = driver.session().run('MATCH (c:{}) RETURN c.{}'.format('Airport', i))
    res = set(res.value())
    return res

In [86]:
options = get_options('iata_code')

In [87]:
table = widgets.Dropdown(
    options=options)

print('Выберите аэропорт, прибывающие рейсы в который Вы хотите узнать: ')
table

Выберите аэропорт, прибывающие рейсы в который Вы хотите узнать: 


Dropdown(options=('SVO', 'TXL', 'VKO', 'GOJ', 'BAX', 'MMK', 'KZN', 'LED', 'TLV', 'MSQ', 'AER', 'TLL', 'DME', '…

In [83]:
airport = table.value
coll_query = "MATCH (a:Airport)-[:flies_to]-(t:Timetable) WHERE a.iata_code = '" + airport + "' RETURN collect(t.flight_number) as t"

flights = driver.session().run(coll_query)

In [74]:
print('Номера рейсов, прибывающих в аэропорт ' + airport + ': ')
for i in flights:
    print(i)

Номера рейсов, прибывающих в аэропорт SVO: 
<Record t=['1317', '2263', '509']>


# 2. Count

Сколько аэропортов в городе? 

In [123]:
cities_toc = ['Antalya',
 'Barcelona',
 'Barnaul',
 'Berlin',
 'Kazan',
 'Minsk',
 'Moscow',
 'Murmansk',
 'Nizhny_Novgorod',
 'Saint_Petersburg',
 'Sochi',
 'Tallinn',
 'Tel_Aviv']

In [124]:
choose_city = widgets.RadioButtons(                        ###чтобы выбрать несколько признаков, зажми CTRL и выбирай
    options = cities_toc)

print('выберите город')

choose_city

выберите город


RadioButtons(options=('Antalya', 'Barcelona', 'Barnaul', 'Berlin', 'Kazan', 'Minsk', 'Moscow', 'Murmansk', 'Ni…

In [128]:
count_query = "MATCH (n { city: '"+choose_city.value+"' })-->(x) RETURN count(x)"

In [129]:
run = driver.session().run(count_query)
counted = run.value()
counted

[3]

# Запросы к удаленным вершинам

### 1. Показать все "исходящие"/"входящие" рейсы, которые отправляются из/в столиц или не столиц.

In [97]:
fl_type = widgets.RadioButtons(
    options = ['входящие', 'исходящие'])

print('Какие рейсы найти?')
fl_type

Какие рейсы найти?


RadioButtons(options=('входящие', 'исходящие'), value='входящие')

In [98]:
city_type = widgets.RadioButtons(
    options = ['столицы', 'не столицы'])

print('Рейсы для каких городов?')
city_type

Рейсы для каких городов?


RadioButtons(options=('столицы', 'не столицы'), value='столицы')

In [105]:
if fl_type.value == 'входящие':
    fl = 'flies_to'
if fl_type.value == 'исходящие':
    fl = 'flies_from'
    
if city_type.value == 'столицы':
    c = '1'
if city_type.value == 'не столицы':
    c = '0'

In [106]:
query = "match (c:City)-[:located_in]-(a:Airport)-[:" + fl + "]-(t:Timetable) where c.capital = '" + c + "' return t.flight_number as t"
dep = driver.session().run(query)
dep.value()

['3661', '845', '2263']

### 2. Показать информацию о самолетах, которые использует определенная авиакомпания.

In [108]:
def get_options(i):
    res = driver.session().run('MATCH (c:{}) RETURN c.{}'.format('Airline', i))
    res = set(res.value())
    return res

In [117]:
options = get_options('name')

air_table = widgets.Dropdown(
    options=options)

print('Выберите авиакомпанию, информацию о самолетах которой вы хотите узнать: ')
air_table

Выберите авиакомпанию, информацию о самолетах которой вы хотите узнать: 


Dropdown(options=('S7 Airlines', 'Ural Airlines', 'Pobeda', 'UVT Aero', 'Red Wings Airlines', 'Aeroflot', 'Bel…

In [111]:
def get_columns(t):
    res = driver.session().run('MATCH (n:{}) RETURN DISTINCT keys(n), size(keys(n)) ORDER BY size(keys(n)) DESC'.format(t))
    res = res.value()[0]
    return res

In [116]:
info_type = get_columns('Aircraft')

table = widgets.Dropdown(
    options=info_type)

print('Выберите тип информации о самолетах: ')
table

Выберите тип информации о самолетах: 


Dropdown(options=('capacity', 'body_type', 'manufacturer', 'distance', 'aircraft_type'), value='capacity')

In [121]:
query = "match (a:Airline)-[:operated_by]-(t:Timetable)-[:aircraft_type]-(ai:Aircraft) where a.name = '" + air_table.value + "' return ai." + table.value + " as ai"
info = driver.session().run(query)
info.value()

['narrow', 'narrow', 'narrow']

### 3. Показать рейсы из точки А в точку Б

In [14]:
def get_options_c(i):
    res = driver.session().run('MATCH (c:{}) RETURN c.{}'.format('City', i))
    res = set(res.value())
    return res

In [16]:
options = get_options('city_name')

out_table = widgets.Dropdown(
    options=options)

print('Выберите город, из которого вы хотите улететь: ')
out_table

Выберите город, из которого вы хотите улететь: 


Dropdown(options=('Moscow', 'Saint_Petersburg', 'Tel_Aviv', 'Kazan', 'Berlin', 'Tallinn', 'Barcelona', 'Barnau…

In [18]:
options = get_options('city_name')

in_table = widgets.Dropdown(
    options=options)

print('Выберите город, в который вы хотите прилететь: ')
in_table

Выберите город, в который вы хотите прилететь: 


Dropdown(options=('Moscow', 'Saint_Petersburg', 'Tel_Aviv', 'Kazan', 'Berlin', 'Tallinn', 'Barcelona', 'Barnau…

In [24]:
query = 'match (c1:City)-[:located_in]-(a1:Airport)-[:flies_from]-(t:Timetable)-[:flies_to]-(a2:Airport)-[:located_in]-(c2:City) where c1.city_name = "' + out_table.value +'" and c2.city_name = "'+ in_table.value +'" return properties(t)'
flight = driver.session().run(query)
flight.value()[0]

{'arrival_time': '15:15:00',
 'departure_airport': 'AER',
 'flight_number': '278',
 'airline': 'U6',
 'airplane_model': 'A 320-200',
 'departure_time': '12:55:00',
 'flight_duration': '02:20:00',
 'destination_airport': 'DME'}